# Rice Classification: Classifying a Grain of Rice with Neutral Physical Properties
### DSCI100: Project Proposal (Group 124)

##### Sean Park (66350992)
##### Deacon Sowerby (50471499)
##### Gabriel Reyes (80274921)
##### David Rao (55832919)

In [5]:
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows=6)

## 1. Introduction

### -> About the dataset
Different types of rice are classified biologically by their origin; however, some of these types can also be identified by their physical shape. In our dataset, "riceClassification.csv", there are more than 18,000 observations of rice grains with specified physical measurements that classify as either the rice type "**Gonen**' (indicated by the factor 0) or "**Jasmine**" (indicated by the factor 1). This data has been specially modified for binary classification.

In [6]:
url = "https://raw.githubusercontent.com/sean-prk/group-124-proposal/main/riceClassification.csv"
# referenced a csv file from our own GitHub repository

download.file(url, "riceClassification.csv")

rice_data <- read_csv("riceClassification.csv") |>
    mutate(Class = as_factor(Class))
rice_data

Rows: 18185 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (12): id, Area, MajorAxisLength, MinorAxisLength, Eccentricity, ConvexAr...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,4537,92.22932,64.01277,0.7199162,4677,76.00452,0.6575362,273.085,0.7645096,1.440796,1
2,2872,74.69188,51.40045,0.7255527,3015,60.47102,0.7130089,208.317,0.8316582,1.453137,1
3,3048,76.29316,52.04349,0.7312109,3132,62.29634,0.7591532,210.012,0.8684336,1.465950,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
18183,6365,154.7771,52.90808,0.9397603,6531,90.02316,0.5612875,342.253,0.6828318,2.925396,0
18184,5960,151.3979,51.47460,0.9404271,6189,87.11204,0.4923992,343.371,0.6352269,2.941216,0
18185,6134,153.0820,51.59061,0.9415000,6283,88.37450,0.4899752,338.613,0.6722741,2.967245,0


### -> Our objective
We will build a classifier that classifies a rice grain as either Gonen or Jasmine given its physical properties. Using this classifier, we want to predict the class of a new rice observation that we will create to be seemingly neutral; the variables of this new observation will be decided by taking the average for that measurement from the original dataset (see below).

In [8]:
new_obs <- rice_data |>
    map(cols = Area:Class, mean) |>  
                #take mean of every potential predictor variable
    as_tibble() 
                #use results as the measurements for new observation
new_obs

Warning message in mean.default(.x[[i]], ...):
“argument is not numeric or logical: returning NA”


id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9093,7036.493,151.6808,59.80785,0.9154064,7225.818,94.13295,0.6166528,351.6069,0.7079983,2.599081,NA


### -> Our question
**Which class, either Gonen or Jasmine, will this "neutral" observation be classified as?**
As well, what statistical factors influence/determine the decision of our classifier?

### -> Our thesis

We expect our "neutral" observation to be classified as Jasmine (1).

We saw that there are more observations classified as Jasmine in the original dataset. Therefore, when taking the average of each variable to set new values for the new observation, the averages would be more influenced by Jasmine observations than Gonen (due to its frequency).

## 2. Methods and Results

### -> Pre-analysis

(steps included for making process, do not need to be in final version.)

##### i) Loading/tidying the data and creating the "new" observation

... 

##### ii) Summary and visualization

(can grab this from proposal)

##### iii) Selecting predictors

### -> Data analysis with steps

(steps included for making process, do not need to be in final version.)

##### i) Splitting the data

##### ii) Creating a recipe / tuning model

##### iii) Tuning the model and choosing the best K

##### iv) Making a model with the best parameter K

##### v) Evaluating the estimated accuracy of the classifier

##### vi) Classifying the "new" observation

## 3.  Discussion

... what we found, if it matches our thesis, statistical impact of findings, possible future questions.

## 4. References

